<a href="https://colab.research.google.com/github/Moot-bot/Routelab/blob/main/Webscrap_Anylogicipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 78.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.7/512.7 kB 32.2 MB/s eta 0:00:00


In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from urllib.parse import urljoin, urlparse
import time
import json
from collections import deque
import os

def setup_driver():
    """Configurar o driver do Chrome"""
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--window-size=1920,1080")

    driver = webdriver.Chrome(options=chrome_options)
    return driver

def get_page_content(driver, url):
    """Extrair conteúdo de uma página específica"""
    try:
        driver.get(url)
        time.sleep(2)  # Esperar carregar

        # Extrair título
        title = driver.title
        if not title or title == "AnyLogic Help":
            try:
                title = driver.find_element(By.TAG_NAME, "h1").text
            except:
                title = "Sem título"

        # Extrair conteúdo principal
        content_selectors = [
            "main", ".content", "#content", ".main-content",
            ".documentation", "article", ".help-content", "body"
        ]

        content_element = None
        for selector in content_selectors:
            try:
                content_element = driver.find_element(By.CSS_SELECTOR, selector)
                break
            except:
                continue

        if content_element:
            content = content_element.text
        else:
            content = driver.find_element(By.TAG_NAME, "body").text

        # Extrair links da página (para continuar o crawling)
        links = []
        try:
            link_elements = driver.find_elements(By.CSS_SELECTOR, "a[href]")
            for link in link_elements:
                href = link.get_attribute("href")
                if href and "anylogic.help" in href:
                    links.append(href)
        except:
            pass

        return {
            "url": url,
            "title": title,
            "content": content,
            "links": links
        }

    except Exception as e:
        print(f"❌ Erro ao acessar {url}: {e}")
        return None

def crawl_anylogic_docs(start_url, max_pages=100):
    """Crawler para navegar por toda a documentação"""
    driver = setup_driver()

    try:
        # Estruturas para controle do crawling
        visited = set()
        to_visit = deque([start_url])
        all_pages = []

        print(f"🚀 Iniciando crawling a partir de: {start_url}")
        print(f"📝 Limite máximo de páginas: {max_pages}")

        while to_visit and len(visited) < max_pages:
            current_url = to_visit.popleft()

            if current_url in visited:
                continue

            print(f"📖 Processando página {len(visited) + 1}: {current_url}")

            # Extrair conteúdo da página atual
            page_data = get_page_content(driver, current_url)

            if page_data:
                visited.add(current_url)
                all_pages.append(page_data)

                # Adicionar novos links para visitar
                for link in page_data["links"]:
                    if link not in visited and link not in to_visit:
                        # Filtrar apenas links relevantes da documentação
                        if any(pattern in link for pattern in [
                            "/anylogic/",
                            "anylogic.help/anylogic",
                            "index.html"
                        ]):
                            to_visit.append(link)

                print(f"✅ Página salva: {page_data['title']}")
                print(f"   📊 Links encontrados: {len(page_data['links'])}")
                print(f"   📏 Conteúdo: {len(page_data['content'])} caracteres")

            # Pequena pausa para não sobrecarregar o servidor
            time.sleep(1)

        print(f"\n🎯 Crawling completo! {len(all_pages)} páginas processadas.")
        return all_pages

    finally:
        driver.quit()

def save_documentation(pages, output_dir="anylogic_documentation"):
    """Salvar a documentação extraída em arquivos organizados"""
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Salvar cada página individualmente
    for i, page in enumerate(pages):
        filename = f"page_{i+1:03d}.txt"
        filepath = os.path.join(output_dir, filename)

        with open(filepath, "w", encoding="utf-8") as f:
            f.write(f"URL: {page['url']}\n")
            f.write(f"Título: {page['title']}\n")
            f.write("=" * 80 + "\n")
            f.write(page['content'])
            f.write("\n" + "=" * 80 + "\n")
            f.write(f"Links encontrados: {len(page['links'])}\n")

    # Salvar índice geral
    index_path = os.path.join(output_dir, "index.json")
    index_data = []

    for i, page in enumerate(pages):
        index_data.append({
            "id": i + 1,
            "title": page["title"],
            "url": page["url"],
            "content_length": len(page["content"]),
            "links_count": len(page["links"]),
            "filename": f"page_{i+1:03d}.txt"
        })

    with open(index_path, "w", encoding="utf-8") as f:
        json.dump(index_data, f, indent=2, ensure_ascii=False)

    # Salvar um arquivo consolidado
    consolidated_path = os.path.join(output_dir, "consolidated_documentation.txt")
    with open(consolidated_path, "w", encoding="utf-8") as f:
        f.write("DOCUMENTAÇÃO COMPLETA ANYLOGIC\n")
        f.write("=" * 50 + "\n\n")

        for i, page in enumerate(pages):
            f.write(f"CAPÍTULO {i+1}: {page['title']}\n")
            f.write(f"URL: {page['url']}\n")
            f.write("-" * 50 + "\n")
            f.write(page['content'])
            f.write("\n\n" + "=" * 80 + "\n\n")

    return len(pages)

# EXECUÇÃO PRINCIPAL
if __name__ == "__main__":
    start_url = "https://anylogic.help/anylogic/index.html"

    print("🌐 INICIANDO CRAWLER DA DOCUMENTAÇÃO ANYLOGIC")
    print("=" * 60)

    # Fazer o crawling de todas as páginas
    all_pages = crawl_anylogic_docs(start_url, max_pages=50)

    if all_pages:
        # Salvar os resultados
        pages_saved = save_documentation(all_pages)

        print(f"\n💾 DOCUMENTAÇÃO SALVA COM SUCESSO!")
        print(f"📁 Pasta: anylogic_documentation/")
        print(f"📄 Total de páginas: {pages_saved}")
        print(f"📊 Arquivos gerados:")
        print(f"   - page_001.txt a page_{pages_saved:03d}.txt (páginas individuais)")
        print(f"   - index.json (índice da documentação)")
        print(f"   - consolidated_documentation.txt (arquivo consolidado)")

        # Mostrar estatísticas
        total_content = sum(len(page['content']) for page in all_pages)
        total_links = sum(len(page['links']) for page in all_pages)

        print(f"\n📈 ESTATÍSTICAS:")
        print(f"   📏 Total de caracteres: {total_content:,}")
        print(f"   🔗 Total de links encontrados: {total_links}")
        print(f"   📚 Média de conteúdo por página: {total_content//len(all_pages):,} caracteres")

    else:
        print("❌ Nenhuma página foi processada!")

🌐 INICIANDO CRAWLER DA DOCUMENTAÇÃO ANYLOGIC
🚀 Iniciando crawling a partir de: https://anylogic.help/anylogic/index.html
📝 Limite máximo de páginas: 50
📖 Processando página 1: https://anylogic.help/anylogic/index.html
✅ Página salva: AnyLogic Help | AnyLogic Help
   📊 Links encontrados: 957
   📏 Conteúdo: 587 caracteres
📖 Processando página 2: https://anylogic.help/anylogic/introduction/index.html
✅ Página salva: Introduction | AnyLogic Help
   📊 Links encontrados: 952
   📏 Conteúdo: 687 caracteres
📖 Processando página 3: https://anylogic.help/anylogic/introduction/activation-index.html
✅ Página salva: AnyLogic activation | AnyLogic Help
   📊 Links encontrados: 948
   📏 Conteúdo: 787 caracteres
📖 Processando página 4: https://anylogic.help/anylogic/introduction/activation-with-permanent-key.html
✅ Página salva: Activating AnyLogic with the key | AnyLogic Help
   📊 Links encontrados: 955
   📏 Conteúdo: 5565 caracteres
📖 Processando página 5: https://anylogic.help/anylogic/introduction/a

In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from urllib.parse import urljoin, urlparse
import time
import json
from collections import deque
import os
import re

def setup_driver():
    """Configurar o driver do Chrome"""
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--window-size=1920,1080")

    driver = webdriver.Chrome(options=chrome_options)
    return driver

def clean_content(text):
    """Limpar e formatar o conteúdo extraído"""
    # Remover linhas em branco excessivas
    lines = [line.strip() for line in text.split('\n') if line.strip()]

    # Juntar linhas relacionadas
    cleaned_lines = []
    for line in lines:
        if cleaned_lines and len(line) < 100 and not line.endswith(('.', ':', ';')):
            # Se a linha é curta e não termina com pontuação, pode ser continuação
            cleaned_lines[-1] += " " + line
        else:
            cleaned_lines.append(line)

    return '\n'.join(cleaned_lines)

def get_page_content(driver, url):
    """Extrair conteúdo de uma página específica"""
    try:
        driver.get(url)
        time.sleep(2)  # Esperar carregar

        # Extrair título
        title = driver.title
        if not title or title == "AnyLogic Help":
            try:
                title = driver.find_element(By.TAG_NAME, "h1").text
            except:
                try:
                    title = driver.find_element(By.CSS_SELECTOR, ".title, .page-title").text
                except:
                    title = "Sem título"

        # Extrair conteúdo principal
        content_selectors = [
            "main", ".content", "#content", ".main-content",
            ".documentation", "article", ".help-content", "body"
        ]

        content_element = None
        for selector in content_selectors:
            try:
                content_element = driver.find_element(By.CSS_SELECTOR, selector)
                break
            except:
                continue

        if content_element:
            content = content_element.text
        else:
            content = driver.find_element(By.TAG_NAME, "body").text

        # Limpar conteúdo
        content = clean_content(content)

        # Extrair links da página (para continuar o crawling)
        links = []
        try:
            link_elements = driver.find_elements(By.CSS_SELECTOR, "a[href]")
            for link in link_elements:
                href = link.get_attribute("href")
                if href and "anylogic.help/anylogic" in href:
                    # Normalizar URL
                    if "#" in href:
                        href = href.split("#")[0]  # Remover âncoras
                    links.append(href)
        except:
            pass

        return {
            "url": url,
            "title": title,
            "content": content,
            "links": list(set(links))  # Remover duplicatas
        }

    except Exception as e:
        print(f"❌ Erro ao acessar {url}: {e}")
        return None

def crawl_anylogic_docs(start_url):
    """Crawler para navegar por TODA a documentação sem limites"""
    driver = setup_driver()

    try:
        # Estruturas para controle do crawling
        visited = set()
        to_visit = deque([start_url])
        all_pages = []

        print(f"🚀 Iniciando crawling a partir de: {start_url}")
        print("📝 SEM LIMITE DE PÁGINAS - Coletando toda a documentação...")

        while to_visit:
            current_url = to_visit.popleft()

            if current_url in visited:
                continue

            print(f"📖 Processando página {len(visited) + 1}: {current_url}")

            # Extrair conteúdo da página atual
            page_data = get_page_content(driver, current_url)

            if page_data:
                visited.add(current_url)
                all_pages.append(page_data)

                # Adicionar novos links para visitar
                new_links_count = 0
                for link in page_data["links"]:
                    if link not in visited and link not in to_visit:
                        # Filtrar apenas links relevantes da documentação
                        if any(pattern in link for pattern in [
                            "/anylogic/",
                            "anylogic.help/anylogic",
                            "index.html"
                        ]) and not any(exclude in link for exclude in [
                            "javascript:", "mailto:", ".pdf", ".zip"
                        ]):
                            to_visit.append(link)
                            new_links_count += 1

                print(f"✅ Página {len(visited)}: {page_data['title'][:50]}...")
                print(f"   📊 Links encontrados: {len(page_data['links'])} | Novos: {new_links_count}")
                print(f"   📏 Conteúdo: {len(page_data['content'])} caracteres")
                print(f"   📚 Fila: {len(to_visit)} páginas restantes")

            # Pequena pausa para não sobrecarregar o servidor
            time.sleep(1)

        print(f"\n🎯 Crawling completo! {len(all_pages)} páginas processadas.")
        return all_pages

    finally:
        driver.quit()

def create_consolidated_documentation(pages, output_file="anylogic_documentacao_completa.txt"):
    """Criar um arquivo único consolidado com toda a documentação"""

    print(f"📄 Criando arquivo consolidado: {output_file}")

    with open(output_file, "w", encoding="utf-8") as f:
        # Cabeçalho
        f.write("=" * 80 + "\n")
        f.write("DOCUMENTAÇÃO COMPLETA ANYLOGIC\n")
        f.write("=" * 80 + "\n")
        f.write(f"Total de páginas: {len(pages)}\n")
        f.write(f"Data de extração: {time.strftime('%d/%m/%Y %H:%M:%S')}\n")
        f.write("=" * 80 + "\n\n")

        # Conteúdo de cada página
        for i, page in enumerate(pages, 1):
            f.write(f"{'=' * 80}\n")
            f.write(f"CAPÍTULO {i:03d}: {page['title']}\n")
            f.write(f"URL: {page['url']}\n")
            f.write(f"{'=' * 80}\n\n")
            f.write(page['content'])
            f.write("\n\n")

            # Progresso
            if i % 10 == 0:
                print(f"📝 Escrita: {i}/{len(pages)} páginas...")

    print(f"✅ Arquivo consolidado criado: {output_file}")
    return output_file

def save_individual_pages(pages, output_dir="anylogic_pages"):
    """Salvar páginas individuais (opcional)"""
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for i, page in enumerate(pages, 1):
        filename = f"pagina_{i:03d}_{re.sub(r'[^a-zA-Z0-9]', '_', page['title'][:30])}.txt"
        filepath = os.path.join(output_dir, filename)

        with open(filepath, "w", encoding="utf-8") as f:
            f.write(f"URL: {page['url']}\n")
            f.write(f"Título: {page['title']}\n")
            f.write("=" * 80 + "\n")
            f.write(page['content'])

    print(f"✅ {len(pages)} páginas individuais salvas em '{output_dir}/'")

def generate_statistics(pages):
    """Gerar estatísticas da documentação"""
    total_chars = sum(len(page['content']) for page in pages)
    total_words = sum(len(page['content'].split()) for page in pages)

    print("\n📊 ESTATÍSTICAS DA DOCUMENTAÇÃO:")
    print(f"   📚 Total de páginas: {len(pages)}")
    print(f"   📏 Total de caracteres: {total_chars:,}")
    print(f"   🔤 Total de palavras: {total_words:,}")
    print(f"   📄 Média por página: {total_chars//len(pages):,} caracteres")
    print(f"   🌐 URLs únicas coletadas: {len(set(page['url'] for page in pages))}")

    # Top 10 páginas com mais conteúdo
    sorted_pages = sorted(pages, key=lambda x: len(x['content']), reverse=True)[:10]
    print(f"\n🏆 TOP 10 PÁGINAS COM MAIS CONTEÚDO:")
    for i, page in enumerate(sorted_pages, 1):
        print(f"   {i:2d}. {page['title'][:50]}... ({len(page['content']):,} chars)")

# EXECUÇÃO PRINCIPAL
if __name__ == "__main__":
    start_url = "https://anylogic.help/anylogic/index.html"

    print("🌐 INICIANDO CRAWLER COMPLETO DA DOCUMENTAÇÃO ANYLOGIC")
    print("=" * 60)
    print("⚡ SEM LIMITES - Coletando TODA a documentação disponível")
    print("💾 Será criado um único arquivo consolidado com todo o conteúdo")
    print("=" * 60)

    # Fazer o crawling de TODAS as páginas
    start_time = time.time()
    all_pages = crawl_anylogic_docs(start_url)
    crawling_time = time.time() - start_time

    if all_pages:
        # Criar arquivo consolidado único
        consolidated_file = create_consolidated_documentation(all_pages)

        # (Opcional) Salvar páginas individuais também
        save_individual_pages(all_pages)

        # Gerar estatísticas
        generate_statistics(all_pages)

        print(f"\n💾 DOCUMENTAÇÃO SALVA COM SUCESSO!")
        print(f"📁 Arquivo principal: {consolidated_file}")
        print(f"⏱️  Tempo total: {crawling_time:.2f} segundos")
        print(f"📦 Tamanho aproximado: {os.path.getsize(consolidated_file) / 1024 / 1024:.2f} MB")

    else:
        print("❌ Nenhuma página foi processada!")

🌐 INICIANDO CRAWLER COMPLETO DA DOCUMENTAÇÃO ANYLOGIC
⚡ SEM LIMITES - Coletando TODA a documentação disponível
💾 Será criado um único arquivo consolidado com todo o conteúdo
🚀 Iniciando crawling a partir de: https://anylogic.help/anylogic/index.html
📝 SEM LIMITE DE PÁGINAS - Coletando toda a documentação...
📖 Processando página 1: https://anylogic.help/anylogic/index.html
✅ Página 1: AnyLogic Help | AnyLogic Help...
   📊 Links encontrados: 332 | Novos: 331
   📏 Conteúdo: 587 caracteres
   📚 Fila: 331 páginas restantes
📖 Processando página 2: https://anylogic.help/anylogic/agentbased/agent-api.html
✅ Página 2: Agent API | AnyLogic Help...
   📊 Links encontrados: 332 | Novos: 0
   📏 Conteúdo: 1561 caracteres
   📚 Fila: 330 páginas restantes
📖 Processando página 3: https://anylogic.help/anylogic/ui/ge-keyboard-shortcuts.html
✅ Página 3: Graphical editor: hotkeys and mouse shortcuts | An...
   📊 Links encontrados: 332 | Novos: 0
   📏 Conteúdo: 2831 caracteres
   📚 Fila: 329 páginas restant